In [ ]:
# Our standard packages for data science.
import os
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import time

# Our main packages for standard machine learning
from sklearn.linear_model import *
from sklearn.tree import *
from sklearn.svm import *
from sklearn.naive_bayes import *
from sklearn.ensemble import *
from sklearn.multiclass import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import StandardScaler, RobustScaler
from scipy.stats import randint as sp_randint
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from scipy import stats
from scipy.stats import norm
from matplotlib import rcParams
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import gc
gc.enable()

print('Packages are ready!')

In [ ]:
winedata = pd.read_csv('Data\winedata.csv')

print('Data is ready!')

In [ ]:
ctarget = winedata['quality']
qtarget = winedata['color']
data = winedata.drop('quality', axis=1)
data = data.drop('color', axis=1)
print('Done!')

In [ ]:
print(winedata.shape)
print(data.shape)
print(qtarget.shape)
print(ctarget.shape)

In [ ]:
from sklearn.datasets import load_boston

boston = load_boston()

In [ ]:
boston.keys()

In [ ]:
boston.data.shape

In [ ]:
print(boston.feature_names)

In [ ]:
print(boston.DESCR)

In [ ]:
bos = pd.DataFrame(boston.data)
bos.head()

In [ ]:
bos.columns = boston.feature_names
bos.head()

In [ ]:
bos['PRICE'] = boston.target
bos.head()

In [ ]:
bos.describe()

In [ ]:
plt.scatter(bos.RM, bos.PRICE)
plt.xlabel("Average Rooms Per Dwelling (RM)")
plt.ylabel("Housing Price")
plt.title("Relationship between RM and Price")

In [ ]:
sns.regplot(y="PRICE", x="RM", data=bos, fit_reg = True)

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
model = ols('PRICE ~ RM',bos).fit()
print(model.summary())

In [ ]:
model = ols('PRICE ~ CRIM',bos).fit()
print(model.summary())

In [ ]:
model = ols('PRICE ~ PTRATIO',bos).fit()
print(model.summary())

In [ ]:
model = ols('PRICE ~ RM+DIS+TAX+B+NOX+CRIM+ZN+INDUS+CHAS+AGE+RAD+PTRATIO+LSTAT',bos).fit()
model.fittedvalues.head()
print(model.summary())

In [ ]:
reduced = ols('PRICE ~ PTRATIO+RM+CRIM+LSTAT',bos).fit()
print(reduced.summary())

In [ ]:
plt.scatter(bos.PRICE, model.fittedvalues)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted prices")
plt.title("Prices vs Predicted Prices")

In [ ]:
from sklearn.linear_model import LinearRegression
X = bos.drop('PRICE', axis = 1)

lm = LinearRegression()
lm.fit(X, bos.PRICE)

In [ ]:
print('Estimated intercept coefficient: {}'.format(lm.intercept_))

In [ ]:
pd.DataFrame({'features': X.columns, 'estimatedCoefficients': lm.coef_})[['features', 'estimatedCoefficients']]

In [ ]:
plt.hist(lm.predict(X))
plt.title('Predicted Housing Prices (fitted values)')
plt.xlabel('Price')
plt.ylabel('Frequency')

In [ ]:
lm = LinearRegression()
lm.fit(X[['PTRATIO']], bos.PRICE)
msePTRATIO = np.mean((bos.PRICE - lm.predict(X[['PTRATIO']])) ** 2)
print(msePTRATIO)

In [ ]:
lm = LinearRegression()
lm.fit(X[['CRIM']], bos.PRICE)
mseCRIM = np.mean((bos.PRICE - lm.predict(X[['CRIM']])) ** 2)
print(mseCRIM)

In [ ]:
lm = LinearRegression()
lm.fit(X[['RM']], bos.PRICE)
mseRM = np.mean((bos.PRICE - lm.predict(X[['RM']])) ** 2)
print(mseRM)

In [ ]:
lm = LinearRegression()
lm.fit(X[['LSTAT']], bos.PRICE)
mseLSTAT = np.mean((bos.PRICE - lm.predict(X[['LSTAT']])) ** 2)
print(mseLSTAT)

In [ ]:
x = data
y = qtarget
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=59)
measured= np.zeros((x.shape[0]))
score = 0

for times, (trn_idx, val_idx) in enumerate(folds.split(x.values,y.values)):
    model = LogisticRegression(n_jobs=-1)
    model.fit(x.iloc[trn_idx],y[trn_idx])
    measured[val_idx] = model.predict(x.iloc[val_idx])
    score += model.score(x.iloc[val_idx],y[val_idx])
    print("Fold: {} score: {}".format(times,model.score(x.iloc[val_idx],y[val_idx])))

    gc.collect()

print('Avg Accuracy RF', score / folds.n_splits)

In [ ]:
x = data
y = qtarget
folds = KFold(n_splits=5, shuffle=True, random_state=59)
measured= np.zeros((x.shape[0]))
score = 0

for times, (trn_idx, val_idx) in enumerate(folds.split(x.values,y.values)):
    model = Lasso()
    model.fit(x.iloc[trn_idx],y[trn_idx])
    measured[val_idx] = model.predict(x.iloc[val_idx])
    score += model.score(x.iloc[val_idx],y[val_idx])
    print("Fold: {} score: {}".format(times,model.score(x.iloc[val_idx],y[val_idx])))

    gc.collect()
    
print('Avg Accuracy RF', score / folds.n_splits)

In [ ]:
x = data
y = qtarget
folds = KFold(n_splits=5, shuffle=True, random_state=59)
measured= np.zeros((x.shape[0]))
score = 0

for times, (trn_idx, val_idx) in enumerate(folds.split(x.values,y.values)):
    model = DecisionTreeClassifier()
    model.fit(x.iloc[trn_idx],y[trn_idx])
    measured[val_idx] = model.predict(x.iloc[val_idx])
    score += model.score(x.iloc[val_idx],y[val_idx])
    print("Fold: {} score: {}".format(times,model.score(x.iloc[val_idx],y[val_idx])))

    gc.collect()
    
print('Avg Accuracy RF', score / folds.n_splits)

In [ ]:
x = data
y = qtarget
folds = KFold(n_splits=5, shuffle=True, random_state=59)
measured= np.zeros((x.shape[0]))
score = 0

for times, (trn_idx, val_idx) in enumerate(folds.split(x.values,y.values)):
    model = LinearSVC()
    model.fit(x.iloc[trn_idx],y[trn_idx])
    measured[val_idx] = model.predict(x.iloc[val_idx])
    score += model.score(x.iloc[val_idx],y[val_idx])
    print("Fold: {} score: {}".format(times,model.score(x.iloc[val_idx],y[val_idx])))

    gc.collect()
    
print('Avg Accuracy RF', score / folds.n_splits)

In [ ]:
x = data
y = qtarget
folds = KFold(n_splits=5, shuffle=True, random_state=59)
measured= np.zeros((x.shape[0]))
score = 0

for times, (trn_idx, val_idx) in enumerate(folds.split(x.values,y.values)):
    model = KNeighborsClassifier()
    model.fit(x.iloc[trn_idx],y[trn_idx])
    measured[val_idx] = model.predict(x.iloc[val_idx])
    score += model.score(x.iloc[val_idx],y[val_idx])
    print("Fold: {} score: {}".format(times,model.score(x.iloc[val_idx],y[val_idx])))

    gc.collect()
    
print('Avg Accuracy RF', score / folds.n_splits)

In [ ]:
x = data
y = qtarget
folds = KFold(n_splits=5, shuffle=True, random_state=59)
measured= np.zeros((x.shape[0]))
score = 0

for times, (trn_idx, val_idx) in enumerate(folds.split(x.values,y.values)):
    model = RandomForestClassifier()
    model.fit(x.iloc[trn_idx],y[trn_idx])
    measured[val_idx] = model.predict(x.iloc[val_idx])
    score += model.score(x.iloc[val_idx],y[val_idx])
    print("Fold: {} score: {}".format(times,model.score(x.iloc[val_idx],y[val_idx])))

    gc.collect()
    
print('Avg Accuracy RF', score / folds.n_splits)

In [ ]:
ccdata = pd.read_csv('https://query.data.world/s/44aoyjg7hb4h64bns6iqnzpa5pmv2c')

In [ ]:
ccdata

In [ ]:
ccdata.isna().sum().max()

In [ ]:
ccdata.describe()

In [ ]:
x = ccdata['Time']

plt.hist(x, bins=100)
plt.ylabel('# of times')
plt.xlabel('Dist of Time')
plt.show()

plt.show()

In [ ]:
x = ccdata[ccdata['Class']==1]['Time']

plt.hist(x, bins=50)
plt.ylabel('# of times')
plt.xlabel('Dist of Time')
plt.show()

plt.show()

In [ ]:
ccdata['scaled_amount'] = RobustScaler().fit_transform(ccdata['Amount'].values.reshape(-1,1))
ccdata['scaled_time'] = RobustScaler().fit_transform(ccdata['Time'].values.reshape(-1,1))

ccdata.drop(['Time','Amount'], axis=1, inplace=True)

scaled_amount = ccdata['scaled_amount']
scaled_time = ccdata['scaled_time']

ccdata.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
ccdata.insert(0, 'scaled_amount', scaled_amount)
ccdata.insert(1, 'scaled_time', scaled_time)

ccdata.head()

In [ ]:
del scaled_amount
del scaled_time

In [ ]:
#When you run this line, you'll need to restart Jupyter Lab.
!pip install --user imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
import imblearn

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report, confusion_matrix

In [ ]:
ccdata = ccdata.sample(frac=1, random_state=42)

fraud_ccdata = ccdata[ccdata['Class'] == 1]
non_fraud_ccdata = ccdata[ccdata['Class'] == 0][:492]

undersample_ccdata = pd.concat([fraud_ccdata, non_fraud_ccdata])

new_ccdata = undersample_ccdata.sample(frac=1, random_state=42)

new_ccdata

In [ ]:
new_cctarget = new_ccdata['Class']
cctarget = ccdata['Class']

new_ccdata = new_ccdata.drop('Class', axis=1)
ccdata = ccdata.drop('Class', axis=1)
print('Data Ready')

In [ ]:
new_ccdata.reset_index(drop=True, inplace=True)
new_cctarget.reset_index(drop=True, inplace=True)

In [ ]:
training_data, test_data, training_target, test_target = train_test_split(ccdata, cctarget, test_size = .1, random_state=42)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(training_data, training_target, test_size = .1, random_state=42)

In [ ]:
sm = SMOTE(sampling_strategy=.01, random_state=42)
cc_smote, cc_smote_target = sm.fit_sample(x_train, y_train)

In [ ]:
cc_smote_target.sum()

In [ ]:
model = LogisticRegression(n_jobs=-1)
model.fit(x_train, y_train)
y_pred = model.predict(test_data)
v_pred = model.predict(x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(x_val, y_val)))
print('Recall: {}'.format(recall_score(y_val, model.predict(x_val))))
print('Precision: {}'.format(precision_score(y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(y_val, v_pred, average='micro')))
print(confusion_matrix(y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(test_data, test_target)))
print('Recall: {}'.format(recall_score(test_target, model.predict(test_data))))
print('Precision: {}'.format(precision_score(test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(test_target, y_pred, average='micro')))
print(confusion_matrix(test_target, y_pred))

In [ ]:
model = LogisticRegression(n_jobs=-1)
model.fit(new_ccdata, new_cctarget)
y_pred = model.predict(test_data)
v_pred = model.predict(x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(x_val, y_val)))
print('Recall: {}'.format(recall_score(y_val, model.predict(x_val))))
print('Precision: {}'.format(precision_score(y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(y_val, v_pred, average='micro')))
print(confusion_matrix(y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(test_data, test_target)))
print('Recall: {}'.format(recall_score(test_target, model.predict(test_data))))
print('Precision: {}'.format(precision_score(test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(test_target, y_pred, average='micro')))
print(confusion_matrix(test_target, y_pred))

In [ ]:
model = LogisticRegression(n_jobs=-1)
model.fit(cc_smote, cc_smote_target)
y_pred = model.predict(test_data)
v_pred = model.predict(x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(x_val, y_val)))
print('Recall: {}'.format(recall_score(y_val, model.predict(x_val))))
print('Precision: {}'.format(precision_score(y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(y_val, v_pred, average='micro')))
print(confusion_matrix(y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(test_data, test_target)))
print('Recall: {}'.format(recall_score(test_target, model.predict(test_data))))
print('Precision: {}'.format(precision_score(test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(test_target, y_pred, average='micro')))
print(confusion_matrix(test_target, y_pred))

In [ ]:
model = LogisticRegression(class_weight= {0:.01, 1:.99},n_jobs=-1)
model.fit(x_train, y_train)
y_pred = model.predict(test_data)
v_pred = model.predict(x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(x_val, y_val)))
print('Recall: {}'.format(recall_score(y_val, model.predict(x_val))))
print('Precision: {}'.format(precision_score(y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(y_val, v_pred, average='micro')))
print(confusion_matrix(y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(test_data, test_target)))
print('Recall: {}'.format(recall_score(test_target, model.predict(test_data))))
print('Precision: {}'.format(precision_score(test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(test_target, y_pred, average='micro')))
print(confusion_matrix(test_target, y_pred))

In [ ]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(test_data)
v_pred = model.predict(x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(x_val, y_val)))
print('Recall: {}'.format(recall_score(y_val, model.predict(x_val))))
print('Precision: {}'.format(precision_score(y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(y_val, v_pred, average='micro')))
print(confusion_matrix(y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(test_data, test_target)))
print('Recall: {}'.format(recall_score(test_target, model.predict(test_data))))
print('Precision: {}'.format(precision_score(test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(test_target, y_pred, average='micro')))
print(confusion_matrix(test_target, y_pred))

In [ ]:
model = RandomForestClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(test_data)
v_pred = model.predict(x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(x_val, y_val)))
print('Recall: {}'.format(recall_score(y_val, model.predict(x_val))))
print('Precision: {}'.format(precision_score(y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(y_val, v_pred, average='micro')))
print(confusion_matrix(y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(test_data, test_target)))
print('Recall: {}'.format(recall_score(test_target, model.predict(test_data))))
print('Precision: {}'.format(precision_score(test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(test_target, y_pred, average='micro')))
print(confusion_matrix(test_target, y_pred))

In [ ]:
model = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(), random_state=0, n_jobs=-1)
model.fit(x_train, y_train)
y_pred = model.predict(test_data)
v_pred = model.predict(x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(x_val, y_val)))
print('Recall: {}'.format(recall_score(y_val, model.predict(x_val))))
print('Precision: {}'.format(precision_score(y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(y_val, v_pred, average='micro')))
print(confusion_matrix(y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(test_data, test_target)))
print('Recall: {}'.format(recall_score(test_target, model.predict(test_data))))
print('Precision: {}'.format(precision_score(test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(test_target, y_pred, average='micro')))
print(confusion_matrix(test_target, y_pred))

In [ ]:
model = BalancedRandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
model.fit(x_train, y_train)
y_pred = model.predict(test_data)
v_pred = model.predict(x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(x_val, y_val)))
print('Recall: {}'.format(recall_score(y_val, model.predict(x_val))))
print('Precision: {}'.format(precision_score(y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(y_val, v_pred, average='micro')))
print(confusion_matrix(y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(test_data, test_target)))
print('Recall: {}'.format(recall_score(test_target, model.predict(test_data))))
print('Precision: {}'.format(precision_score(test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(test_target, y_pred, average='micro')))
print(confusion_matrix(test_target, y_pred))

In [ ]:
w_training_data, w_test_data, q_training_target, q_test_target = train_test_split(data, qtarget, test_size = .2, random_state=42)
w_x_train, w_x_val, q_y_train, q_y_val = train_test_split(w_training_data, q_training_target, test_size = .1, random_state=42)

#### Due to differences in machine resources, your times will differ from the screen and the lecture.
#### Several scripts in this lecture will take a while to run. 
#### It will use a lot of resources and appear to make your machine crash.
#### Just be patient, and check on it's progress periodically.

In [ ]:
#Basic Logistic Regression
start = time.time()

x = data
y = qtarget
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=59)
measured= np.zeros((x.shape[0]))
score = 0

for times, (trn_idx, val_idx) in enumerate(folds.split(x.values,y.values)):
    model = LogisticRegression(n_jobs=-1)
    model.fit(x.iloc[trn_idx],y[trn_idx])
    measured[val_idx] = model.predict(x.iloc[val_idx])
    score += model.score(x.iloc[val_idx],y[val_idx])
    print("Fold: {} score: {}".format(times,model.score(x.iloc[val_idx],y[val_idx])))

    gc.collect()

print('Avg Accuracy RF', score / folds.n_splits)

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
#Basic Lasso
start = time.time()

x = data
y = qtarget
folds = KFold(n_splits=5, shuffle=True, random_state=59)
measured= np.zeros((x.shape[0]))
score = 0

for times, (trn_idx, val_idx) in enumerate(folds.split(x.values,y.values)):
    model = Lasso()
    model.fit(x.iloc[trn_idx],y[trn_idx])
    measured[val_idx] = model.predict(x.iloc[val_idx])
    score += model.score(x.iloc[val_idx],y[val_idx])
    print("Fold: {} score: {}".format(times,model.score(x.iloc[val_idx],y[val_idx])))

    gc.collect()
    
print('Avg Accuracy RF', score / folds.n_splits)

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
#Basic Decision Tree
start = time.time()

x = data
y = qtarget
folds = KFold(n_splits=5, shuffle=True, random_state=59)
measured= np.zeros((x.shape[0]))
score = 0

for times, (trn_idx, val_idx) in enumerate(folds.split(x.values,y.values)):
    model = DecisionTreeClassifier()
    model.fit(x.iloc[trn_idx],y[trn_idx])
    measured[val_idx] = model.predict(x.iloc[val_idx])
    score += model.score(x.iloc[val_idx],y[val_idx])
    print("Fold: {} score: {}".format(times,model.score(x.iloc[val_idx],y[val_idx])))

    gc.collect()
    
print('Avg Accuracy RF', score / folds.n_splits)

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
#Basic Linear SVC
start = time.time()

x = data
y = qtarget
folds = KFold(n_splits=5, shuffle=True, random_state=59)
measured= np.zeros((x.shape[0]))
score = 0

for times, (trn_idx, val_idx) in enumerate(folds.split(x.values,y.values)):
    model = LinearSVC()
    model.fit(x.iloc[trn_idx],y[trn_idx])
    measured[val_idx] = model.predict(x.iloc[val_idx])
    score += model.score(x.iloc[val_idx],y[val_idx])
    print("Fold: {} score: {}".format(times,model.score(x.iloc[val_idx],y[val_idx])))

    gc.collect()
    
print('Avg Accuracy RF', score / folds.n_splits)

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
#Basic K Nearest Neighbor
start = time.time()

x = data
y = qtarget
folds = KFold(n_splits=5, shuffle=True, random_state=59)
measured= np.zeros((x.shape[0]))
score = 0

for times, (trn_idx, val_idx) in enumerate(folds.split(x.values,y.values)):
    model = KNeighborsClassifier()
    model.fit(x.iloc[trn_idx],y[trn_idx])
    measured[val_idx] = model.predict(x.iloc[val_idx])
    score += model.score(x.iloc[val_idx],y[val_idx])
    print("Fold: {} score: {}".format(times,model.score(x.iloc[val_idx],y[val_idx])))

    gc.collect()
    
print('Avg Accuracy RF', score / folds.n_splits)

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
#Basic Random Forest
start = time.time()

x = data
y = qtarget
folds = KFold(n_splits=5, shuffle=True, random_state=59)
measured= np.zeros((x.shape[0]))
score = 0

for times, (trn_idx, val_idx) in enumerate(folds.split(x.values,y.values)):
    model = RandomForestClassifier(n_jobs=-1)
    model.fit(x.iloc[trn_idx],y[trn_idx])
    measured[val_idx] = model.predict(x.iloc[val_idx])
    score += model.score(x.iloc[val_idx],y[val_idx])
    print("Fold: {} score: {}".format(times,model.score(x.iloc[val_idx],y[val_idx])))

    gc.collect()
    
print('Avg Accuracy RF', score / folds.n_splits)

end = time.time()
print('It took {} seconds'.format(end - start))

Performance Grid 1

| **Model**                 | **Fold 0 Score** | **Fold 1 Score** | **Fold 2 Score** | **Fold 3 Score** | **Fold 4 Score** | **Avg Accuracy RF** | **Run Time (s)** |
|---------------------------|------------------|------------------|------------------|------------------|------------------|---------------------|------------------|
| Basic Logistic Regression | 0.9838           | 0.9808           | 0.9838           | 0.9777           | 0.9838           | 0.9820              | 9.1450           |
| Basic Lasso               | 0.4431           | 0.5150           | 0.4881           | 0.4896           | 0.5014           | 0.4874              | 0.4470           |
| Basic Decision Tree       | 0.9854           | 0.9885           | 0.9838           | 0.9900           | 0.9861           | 0.9868              | 0.5620           |
| Basic Linear SVC          | 0.6462           | 0.9823           | 0.9808           | 0.9861           | 0.9838           | 0.9158              | 1.2850           |
| Basic K Nearest Neighbor  | 0.9385           | 0.9485           | 0.9453           | 0.9346           | 0.9453           | 0.9424              | 0.9460           |
| Basic Random Forest       | 0.9900           | 0.9962           | 0.9962           | 0.9954           | 0.9946           | 0.9945              | 3.1260           |




In [ ]:
# Sci Kit Learn's Grid Search for Decision Tree classifier

start = time.time()

parameters = {
    'max_depth': [80, 120, None],
    'max_features': ['auto', None],
    'min_samples_leaf': [3, 25, 50],
    'min_samples_split': [8, 10, 12]
}
model = GridSearchCV(DecisionTreeClassifier(), parameters, scoring='accuracy', cv=5, n_jobs=-1)
model.fit(data, qtarget)
end = time.time()
print('It took {} seconds'.format(end - start))
model.best_estimator_

In [ ]:
# Sci Kit Learn's Grid Search for Logistic Regression

start = time.time()

parameters = {
    'tol': [1e-4, 1e-5, 1e-3],
    'C': [.001, .01, .1, 1, 10, 100],
    'max_iter': [100, 200, 50]
}
model = GridSearchCV(LogisticRegression(), parameters, scoring='accuracy', cv=5, n_jobs=-1)
model.fit(data, qtarget)
end = time.time()
print('It took {} seconds'.format(end - start))
model.best_estimator_

In [ ]:
# Sci Kit Learn's Grid Search for Random Forest
#This script could take an hour or more to exectute
start = time.time()

parameters = {
    'criterion': ['gini','entropy'],
    'bootstrap': [True, False],
    'max_depth': [120, None],
    'max_features': ['auto', None],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 4, 8],
    'n_estimators': [100, 500, 1000]
}
model = GridSearchCV(RandomForestClassifier(), parameters, scoring='accuracy', cv=3, n_jobs=-1)
model.fit(data, qtarget)
end = time.time()
print('It took {} seconds'.format(end - start))
model.best_estimator_


In [ ]:
# Our custom grid search for Random Forest
#This script takes a long time to run.
start = time.time()

grid = {
    'criterion': ['gini','entropy'],
    'bootstrap': [True, False],
    'max_depth': [120, None],
    'max_features': ['auto', None],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 4, 8],
    'n_estimators': [100, 500, 1000],
    'n_jobs': [-1]
}

rf = RandomForestClassifier()
best_score = 0

for g in ParameterGrid(grid):
    rf.set_params(**g)
    rf.fit(w_x_train, q_y_train)
    if rf.score(w_test_data, q_test_target) > best_score:
        best_score = rf.score(w_test_data, q_test_target)
        best_grid = g

end = time.time()
print('It took {} seconds'.format(end - start))
best_grid

In [ ]:
# Our custom grid search for Decision Trees
start = time.time()

grid = {
    'max_depth': [80, 120, None],
    'max_features': ['auto', None],
    'min_samples_leaf': [3, 25, 50],
    'min_samples_split': [8, 10, 12]
}

dt = DecisionTreeClassifier()
best_score = 0

for g in ParameterGrid(grid):
    dt.set_params(**g)
    dt.fit(w_x_train, q_y_train)
    if dt.score(w_test_data, q_test_target) > best_score:
        best_score = dt.score(w_test_data, q_test_target)
        best_grid = g

end = time.time()
print('It took {} seconds'.format(end - start))
best_grid

In [ ]:
# Our custom grid search for Logistic Regression
start = time.time()

grid = {
    'tol': [1e-4, 1e-5, 1e-3],
    'C': [.001, .01, .1, 1, 10, 100],
    'max_iter': [100, 200, 50],
    'n_jobs': [-1]
}

lg = LogisticRegression()
best_score = 0

for g in ParameterGrid(grid):
    lg.set_params(**g)
    lg.fit(w_x_train, q_y_train)
    if lg.score(w_test_data, q_test_target) > best_score:
        best_score = lg.score(w_test_data, q_test_target)
        best_grid = g

end = time.time()
print('It took {} seconds'.format(end - start))
best_grid

In [ ]:
#Our baseline random forest classifier
start = time.time()

model = RandomForestClassifier(n_jobs=-1)
model.fit(w_x_train, q_y_train)
y_pred = model.predict(w_test_data)
v_pred = model.predict(w_x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(w_x_val, q_y_val)))
print('Recall: {}'.format(recall_score(q_y_val, model.predict(w_x_val))))
print('Precision: {}'.format(precision_score(q_y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(q_y_val, v_pred, average='micro')))
print(confusion_matrix(q_y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(w_test_data, q_test_target)))
print('Recall: {}'.format(recall_score(q_test_target, model.predict(w_test_data))))
print('Precision: {}'.format(precision_score(q_test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(q_test_target, y_pred, average='micro')))
print(confusion_matrix(q_test_target, y_pred))

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
#Our custom grid search random forest classifier.
start = time.time()

# These parameters came from the grid search's output.
model = RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=120, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
model.fit(w_x_train, q_y_train)
y_pred = model.predict(w_test_data)
v_pred = model.predict(w_x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(w_x_val, q_y_val)))
print('Recall: {}'.format(recall_score(q_y_val, model.predict(w_x_val))))
print('Precision: {}'.format(precision_score(q_y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(q_y_val, v_pred, average='micro')))
print(confusion_matrix(q_y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(w_test_data, q_test_target)))
print('Recall: {}'.format(recall_score(q_test_target, model.predict(w_test_data))))
print('Precision: {}'.format(precision_score(q_test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(q_test_target, y_pred, average='micro')))
print(confusion_matrix(q_test_target, y_pred))

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
#SKLearn's grid search random forest classifier.
start = time.time()

# These parameters came from the grid search's output.
model = RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
model.fit(w_x_train, q_y_train)
y_pred = model.predict(w_test_data)
v_pred = model.predict(w_x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(w_x_val, q_y_val)))
print('Recall: {}'.format(recall_score(q_y_val, model.predict(w_x_val))))
print('Precision: {}'.format(precision_score(q_y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(q_y_val, v_pred, average='micro')))
print(confusion_matrix(q_y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(w_test_data, q_test_target)))
print('Recall: {}'.format(recall_score(q_test_target, model.predict(w_test_data))))
print('Precision: {}'.format(precision_score(q_test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(q_test_target, y_pred, average='micro')))
print(confusion_matrix(q_test_target, y_pred))

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
#Baseline decision tree classifier.
start = time.time()

model = DecisionTreeClassifier()
model.fit(w_x_train, q_y_train)
y_pred = model.predict(w_test_data)
v_pred = model.predict(w_x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(w_x_val, q_y_val)))
print('Recall: {}'.format(recall_score(q_y_val, model.predict(w_x_val))))
print('Precision: {}'.format(precision_score(q_y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(q_y_val, v_pred, average='micro')))
print(confusion_matrix(q_y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(w_test_data, q_test_target)))
print('Recall: {}'.format(recall_score(q_test_target, model.predict(w_test_data))))
print('Precision: {}'.format(precision_score(q_test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(q_test_target, y_pred, average='micro')))
print(confusion_matrix(q_test_target, y_pred))

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
#Custom decision tree classifier.
start = time.time()

# These parameters came from the grid search's output.
model = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
model.fit(w_x_train, q_y_train)
y_pred = model.predict(w_test_data)
v_pred = model.predict(w_x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(w_x_val, q_y_val)))
print('Recall: {}'.format(recall_score(q_y_val, model.predict(w_x_val))))
print('Precision: {}'.format(precision_score(q_y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(q_y_val, v_pred, average='micro')))
print(confusion_matrix(q_y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(w_test_data, q_test_target)))
print('Recall: {}'.format(recall_score(q_test_target, model.predict(w_test_data))))
print('Precision: {}'.format(precision_score(q_test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(q_test_target, y_pred, average='micro')))
print(confusion_matrix(q_test_target, y_pred))

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
#Custom logistic  regression
start = time.time()

# These parameters came from the grid search's output.
model = LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
model.fit(w_x_train, q_y_train)
y_pred = model.predict(w_test_data)
v_pred = model.predict(w_x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(w_x_val, q_y_val)))
print('Recall: {}'.format(recall_score(q_y_val, model.predict(w_x_val))))
print('Precision: {}'.format(precision_score(q_y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(q_y_val, v_pred, average='micro')))
print(confusion_matrix(q_y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(w_test_data, q_test_target)))
print('Recall: {}'.format(recall_score(q_test_target, model.predict(w_test_data))))
print('Precision: {}'.format(precision_score(q_test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(q_test_target, y_pred, average='micro')))
print(confusion_matrix(q_test_target, y_pred))

end = time.time()
print('It took {} seconds'.format(end - start))

Performance Grid 2

| **Model**                                                 | **Accuracy** | **Recall** | **Precision** | **f1** | **TP** | **FP** | **FN** | **TN** | **Run Time** |
|-----------------------------------------------------------|--------------|------------|---------------|--------|--------|--------|--------|--------|--------------|
| baseline random forest classifier validation              | 0.9981       | 1.0000     | 0.9981        | 0.9981 | 120    | 1      | 0      | 399    | 0.9370       |
| baseline random forest classifier test                    | 0.9938       | 0.9990     | 0.9938        | 0.9938 | 334    | 7      | 1      | 958    | 0.9370       |
| custom grid search random forest classifier validation    | 0.9981       | 1.0000     | 0.9981        | 0.9981 | 120    | 1      | 0      | 399    | 3.7790       |
| custom grid search random forest classifier test          | 0.9962       | 0.9990     | 0.9962        | 0.9962 | 337    | 4      | 1      | 958    | 3.7790       |
| SKLearn's grid search random forest classifier validation | 0.9981       | 1.0000     | 0.9981        | 0.9981 | 120    | 1      | 0      | 399    | 2.0210       |
| SKLearn's grid search random forest classifier test       | 0.9962       | 0.9990     | 0.9962        | 0.9962 | 337    | 4      | 1      | 958    | 2.0210       |
| baseline decision tree classifier validation              | 0.9904       | 0.9950     | 0.9904        | 0.9904 | 118    | 3      | 2      | 397    | 0.0620       |
| baseline decision tree classifier test                    | 0.9831       | 0.9906     | 0.9831        | 0.9831 | 328    | 13     | 9      | 950    | 0.0620       |
| Custom decision tree classifier validation                | 0.9904       | 0.9950     | 0.9904        | 0.9904 | 118    | 3      | 2      | 397    | 0.0510       |
| Custom decision tree classifier test                      | 0.9815       | 0.9927     | 0.9815        | 0.9815 | 324    | 17     | 7      | 952    | 0.0510       |
| Custom logistic regression validation                     | 0.9865       | 0.9875     | 0.9865        | 0.9865 | 119    | 2      | 5      | 394    | 0.2560       |
| Custom logistic regression test                           | 0.9808       | 0.9875     | 0.9808        | 0.9808 | 328    | 13     | 12     | 947    | 0.2560       |


### The following models are run with the credit card data.

In [ ]:
# Baseline Credit Card Data Decision Tree Classifier.
start = time.time()

model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(test_data)
v_pred = model.predict(x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(x_val, y_val)))
print('Recall: {}'.format(recall_score(y_val, model.predict(x_val))))
print('Precision: {}'.format(precision_score(y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(y_val, v_pred, average='micro')))
print(confusion_matrix(y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(test_data, test_target)))
print('Recall: {}'.format(recall_score(test_target, model.predict(test_data))))
print('Precision: {}'.format(precision_score(test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(test_target, y_pred, average='micro')))
print(confusion_matrix(test_target, y_pred))

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
# Decision Tree Classifier with classes weighted .2:.8
start = time.time()

model = DecisionTreeClassifier(class_weight= {0:.2, 1:.8})
model.fit(x_train, y_train)
y_pred = model.predict(test_data)
v_pred = model.predict(x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(x_val, y_val)))
print('Recall: {}'.format(recall_score(y_val, model.predict(x_val))))
print('Precision: {}'.format(precision_score(y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(y_val, v_pred, average='micro')))
print(confusion_matrix(y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(test_data, test_target)))
print('Recall: {}'.format(recall_score(test_target, model.predict(test_data))))
print('Precision: {}'.format(precision_score(test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(test_target, y_pred, average='micro')))
print(confusion_matrix(test_target, y_pred))

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
# Random Forest Classifier with Smote data
#this is a long running script.
start = time.time()

model = RandomForestClassifier()
model.fit(cc_smote, cc_smote_target)
y_pred = model.predict(test_data)
v_pred = model.predict(x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(x_val, y_val)))
print('Recall: {}'.format(recall_score(y_val, model.predict(x_val))))
print('Precision: {}'.format(precision_score(y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(y_val, v_pred, average='micro')))
print(confusion_matrix(y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(test_data, test_target)))
print('Recall: {}'.format(recall_score(test_target, model.predict(test_data))))
print('Precision: {}'.format(precision_score(test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(test_target, y_pred, average='micro')))
print(confusion_matrix(test_target, y_pred))

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
# Random Forest Classifier with classes weighted .2:.8 and smote data
# this is a long running script.
start = time.time()

model = RandomForestClassifier(class_weight= {0:.2, 1:.8})
model.fit(cc_smote, cc_smote_target)
y_pred = model.predict(test_data)
v_pred = model.predict(x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(x_val, y_val)))
print('Recall: {}'.format(recall_score(y_val, model.predict(x_val))))
print('Precision: {}'.format(precision_score(y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(y_val, v_pred, average='micro')))
print(confusion_matrix(y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(test_data, test_target)))
print('Recall: {}'.format(recall_score(test_target, model.predict(test_data))))
print('Precision: {}'.format(precision_score(test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(test_target, y_pred, average='micro')))
print(confusion_matrix(test_target, y_pred))

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
# Random Forest Classifier with smote data and 1,000 estimators and bootstrap False
# This is a long running script.
start = time.time()

model = RandomForestClassifier(n_estimators=1000, bootstrap=False, n_jobs=-1)
model.fit(cc_smote, cc_smote_target)
y_pred = model.predict(test_data)
v_pred = model.predict(x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(x_val, y_val)))
print('Recall: {}'.format(recall_score(y_val, model.predict(x_val))))
print('Precision: {}'.format(precision_score(y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(y_val, v_pred, average='micro')))
print(confusion_matrix(y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(test_data, test_target)))
print('Recall: {}'.format(recall_score(test_target, model.predict(test_data))))
print('Precision: {}'.format(precision_score(test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(test_target, y_pred, average='micro')))
print(confusion_matrix(test_target, y_pred))

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
# Random Forest Classifier with the random undersampled data
start = time.time()

model = RandomForestClassifier()
model.fit(new_ccdata, new_cctarget)
y_pred = model.predict(test_data)
v_pred = model.predict(x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(x_val, y_val)))
print('Recall: {}'.format(recall_score(y_val, model.predict(x_val))))
print('Precision: {}'.format(precision_score(y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(y_val, v_pred, average='micro')))
print(confusion_matrix(y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(test_data, test_target)))
print('Recall: {}'.format(recall_score(test_target, model.predict(test_data))))
print('Precision: {}'.format(precision_score(test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(test_target, y_pred, average='micro')))
print(confusion_matrix(test_target, y_pred))

end = time.time()
print('It took {} seconds'.format(end - start))

In [ ]:
# Custom Grid search to find the best random forest model with the undersampled data.
#This is a long running script.
start = time.time()

grid = {
    'criterion': ['gini','entropy'],
    'bootstrap': [True, False],
    'max_depth': [120, None],
    'max_features': ['auto', None],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 4, 8],
    'n_estimators': [100, 500, 1000],
    'n_jobs': [-1]
}

rf = RandomForestClassifier()
best_score = 0

for g in ParameterGrid(grid):
    rf.set_params(**g)
    rf.fit(new_ccdata, new_cctarget)
    # save if best
    if rf.score(test_data, test_target) > best_score:
        best_score = rf.score(test_data, test_target)
        best_grid = g

end = time.time()
print('It took {} seconds'.format(end - start))
best_grid

In [ ]:
# Our best custom random forest model for the undersampled dataset. Also our best model for this data! 
start = time.time()

model = RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=6, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
model.fit(new_ccdata, new_cctarget)
y_pred = model.predict(test_data)
v_pred = model.predict(x_val)

print('Validation Results')
print('Accuracy: {}'.format(model.score(x_val, y_val)))
print('Recall: {}'.format(recall_score(y_val, model.predict(x_val))))
print('Precision: {}'.format(precision_score(y_val, v_pred, average='micro')))
print('f1: {}'.format(f1_score(y_val, v_pred, average='micro')))
print(confusion_matrix(y_val, v_pred))

print('\nTest Results')
print('Accuracy: {}'.format(model.score(test_data, test_target)))
print('Recall: {}'.format(recall_score(test_target, model.predict(test_data))))
print('Precision: {}'.format(precision_score(test_target, y_pred, average='micro')))
print('f1: {}'.format(f1_score(test_target, y_pred, average='micro')))
print(confusion_matrix(test_target, y_pred))

end = time.time()
print('It took {} seconds'.format(end - start))

Performance Grid 3

| **Model**                                                                                    | **Accuracy** | **Recall** | **Precision** | **f1** | **TP** | **FP** | **FN** | **TN** | **Run Time** |
|----------------------------------------------------------------------------------------------|--------------|------------|---------------|--------|--------|--------|--------|--------|--------------|
| Baseline Credit Card Data Decision Tree Classifier validation                                | 0.9991       | 0.7963     | 0.9991        | 0.9991 | 25567  | 12     | 11     | 43     | 23.012       |
| Baseline Credit Card Data Decision Tree Classifier test                                      | 0.9992       | 0.8148     | 0.9992        | 0.9992 | 28413  | 14     | 10     | 44     | 23.012       |
| Decision Tree Classifier with classes weighted .2:.8 validation                              | 0.9991       | 0.8148     | 0.9991        | 0.9991 | 25567  | 12     | 10     | 44     | 20.489       |
| Decision Tree Classifier with classes weighted .2:.8 test                                    | 0.9994       | 0.8333     | 0.9994        | 0.9994 | 28420  | 7      | 9      | 45     | 20.489       |
| Random Forest Classifier with Smote data validation                                          | 0.9995       | 0.8333     | 0.9995        | 0.9995 | 25574  | 5      | 9      | 45     | 228.916      |
| Random Forest Classifier with Smote data test                                                | 0.9997       | 0.9074     | 0.9997        | 0.9997 | 28423  | 4      | 5      | 49     | 228.916      |
| Random Forest Classifier with classes weighted .2:.8 and smote data validation               | 0.9995       | 0.8333     | 0.9995        | 0.9995 | 25575  | 4      | 9      | 45     | 203.673      |
| Random Forest Classifier with classes weighted .2:.8 and smote data test                     | 0.9997       | 0.8889     | 0.9997        | 0.9997 | 28424  | 3      | 6      | 48     | 203.673      |
| Random Forest Classifier with smote data and 1,000 estimators and bootstrap False validation | 0.9995       | 0.8333     | 0.9995        | 0.9995 | 25575  | 4      | 9      | 45     | 952.100      |
| Random Forest Classifier with smote data and 1,000 estimators and bootstrap False test       | 0.9997       | 0.9074     | 0.9997        | 0.9997 | 28423  | 4      | 5      | 49     | 952.100      |
| Random Forest Classifier with the random undersampled data validation                        | 0.9774       | 1.0000     | 0.9774        | 0.9774 | 25000  | 579    | 0      | 54     | 2.097        |
| Random Forest Classifier with the random undersampled data test                              | 0.9759       | 1.0000     | 0.9759        | 0.9759 | 27742  | 685    | 0      | 54     | 2.097        |
| Our best custom random forest model for the undersampled dataset validation                  | 0.9827       | 1.0000     | 0.9827        | 0.9827 | 25136  | 443    | 0      | 54     | 6.739        |
| Our best custom random forest model for the undersampled dataset test                        | 0.9824       | 0.9815     | 0.9824        | 0.9824 | 27926  | 501    | 1      | 53     | 6.739        |